In [ ]:
pip install opencv-python


In [ ]:
pip install mediapipe

In [ ]:
pip install --user numpy --upgrade

In [ ]:
import numpy as np
import cv2
import mediapipe as mp
import time
from numpy import linalg
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:
def are_finger_stretched(point1, point2, point3):
    outcome = 0
         #
    distance1 = np.linalg.norm((point2 - point1), ord=2)
    distance2 = np.linalg.norm((point3 - point1), ord=2)
    if distance2 > distance1:
        outcome = 1
    
    print(outcome, '\n')
    return outcome

In [ ]:
def detect_gesture(result):
    if (result[0] == 1) and (result[1] == 0) and (result[2] == 0) and (result[3] == 0) and (result[4] == 0):
        gesture = "good"
    elif (result[0] == 1) and (result[1] == 1)and (result[2] == 0) and (result[3] == 0) and (result[4] == 0):
        gesture = "full barre"
    elif (result[0] == 0) and (result[1] == 1)and (result[2] == 0) and (result[3] == 0) and (result[4] == 0):
        gesture = "one"
    elif (result[0] == 0) and (result[1] == 0)and (result[2] == 1) and (result[3] == 0) and (result[4] == 0):
        gesture = "VERY BAD"
    elif (result[0] == 0) and (result[1] == 1)and (result[2] == 1) and (result[3] == 0) and (result[4] == 0):
        gesture = "two"
    elif (result[0] == 1) and (result[1] == 1)and (result[2] == 1) and (result[3] == 0) and (result[4] == 0):
        gesture = "3 finger picking"
    elif (result[0] == 0) and (result[1] == 1)and (result[2] == 1) and (result[3] == 1) and (result[4] == 0):
        gesture = "three"
    elif (result[0] == 0) and (result[1] == 1)and (result[2] == 1) and (result[3] == 1) and (result[4] == 1):
        gesture = "four"
    elif (result[0] == 1) and (result[1] == 1)and (result[2] == 1) and (result[3] == 1) and (result[4] == 1):
        gesture = "five"
    elif (result[0] == 1) and (result[1] == 0)and (result[2] == 0) and (result[3] == 0) and (result[4] == 1):
        gesture = "Shaka"
    elif (result[0] == 0) and (result[1] == 0)and (result[2] == 1) and (result[3] == 1) and (result[4] == 1):
        gesture = "OK or plectrum"
    elif(result[0] == 0) and (result[1] == 0) and (result[2] == 0) and (result[3] == 0) and (result[4] == 0):
        gesture = "closed"
    else:
        gesture = "unknown/not detected"
    
    return gesture

In [ ]:
cap = cv2.VideoCapture(0)

hands = mp.solutions.hands.Hands(static_image_mode=False,
                         max_num_hands=2,
                         min_tracking_confidence=0.5,
                         min_detection_confidence=0.5)

mpDraw = mp.solutions.drawing_utils

previousTime = 0
currentTime = 0

# Array for 5 fingers as figure
figure = [0 for element in range(5)]

# Array for 0-20 trackpoints and 2 hands
landmark = np.empty((21, 2))

while True:
    _, img = cap.read()
    result = hands.process(img)
    #print(result)
    
    if result.multi_hand_landmarks:
        for handsnumber, lms in enumerate(result.multi_hand_landmarks):
            mpDraw.draw_landmarks(img, lms, mp.solutions.hands.HAND_CONNECTIONS)
        
            for id, lm in enumerate(lms.landmark):
        #for id, lm in enumerate(result.multi_hand_landmarks[0].landmark): ONLY FOR ONE HAND WITHOUT SECOND for LOOP

                h, w, _ = img.shape
                cx = int(lm.x * w) 
                cy = int(lm.y * h)
                cv2.circle(img, (cx, cy), 3, (128, 128, 128))
                
                landmark_ = [cx, cy]
                landmark[id,:] = landmark_
                #print(id)
                #print(lm)
            
            #Marks the top of fingers with different colors and thickness
                if id == 4:
                    cv2.circle(img, (cx, cy), 6, (0, 0, 0), cv2.FILLED)
                if id == 8:
                    cv2.circle(img, (cx, cy), 12, (255, 0, 0), cv2.FILLED)
                if id == 12:
                    cv2.circle(img, (cx, cy), 12, (0, 255, 0), cv2.FILLED)
                if id == 16:
                    cv2.circle(img, (cx, cy), 12, (0, 0, 255), cv2.FILLED)
                if id == 20:
                    cv2.circle(img, (cx, cy), 12, (255, 0, 255), cv2.FILLED)
                    
            for i in range(5):
                if i == 0:
                    #pinky
                    figure_ = are_finger_stretched(landmark[17],landmark[4*i+2],landmark[4*i+4])
                else:
                    #wrest
                    figure_ = are_finger_stretched(landmark[0],landmark[4*i+2],landmark[4*i+4])
                    
                figure[i] = figure_
            #print(figure, '\n')
            
            gesture_solution = detect_gesture(figure)
            cv2.putText(img, f"{gesture_solution}", (240, 35*(handsnumber+1)), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 5)

            
    #Shows Frames per Second
        currentTime = time.time()
        fps = 1/(currentTime-previousTime)
        previousTime = currentTime        
        cv2.putText(img, str(int(fps)), (10, 35), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3) #Pos,Font,Scale,Color,Thick
            
        cv2.imshow("Hand Tracking", img) #Name of Window and the Live-Image
        
    if cv2.waitKey(1) == ord('q') : 
        #quit with Q
        break
        
cap.release()
cv2.destroyAllWindows()